In [ ]:
!pip install transformers
!pip install -U -q PyDrive

In [ ]:
!pip install simpletransformers
!pip freeze | grep simpletransformers

Tamil Data

In [3]:
import pandas as pd
df=pd.read_csv('/content/tam_3_train.tsv', delimiter='\t') 
df=df[["text", "category"]]
df = df.dropna()
df_dev=pd.read_csv('/content/tam_3_dev.tsv', delimiter='\t') 
df_dev=df_dev[["text", "category"]]
df_dev = df_dev.dropna()
df_test=pd.read_csv('/content/tam_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'text'}) 
df_test = df_test.dropna()

In [5]:
tedf=pd.read_csv('/content/eng_tam_train.tsv', delimiter='\t') 
tedf=tedf[["text", "category"]]
tedf = tedf.dropna()
tedf_dev=pd.read_csv('/content/eng_tam_dev.tsv', delimiter='\t') 
tedf_dev=tedf_dev[["text", "category"]]
tedf_dev = tedf_dev.dropna()
tedf_test=pd.read_csv('/content/eng_tam_test.tsv', delimiter='\t') 
tedf_test = tedf_test.dropna()

### Mixing english and tamil translated to tamil data

In [7]:
train = pd.concat([df, tedf])
valid = pd.concat([df_dev,tedf_dev])
test = pd.concat([tedf_test,df_test])

In [8]:
print(len(train))
print(len(valid))
print(len(test))

5817
1456
991


In [9]:
concatenated = pd.concat([train, test,valid])

import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

df = concatenated

X = df['text']
y = df['category']

testsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)

train_val_idx, test_idx = next(testsplit.split(X, y))

X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]


validsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

train_idx, val_idx = next(validsplit.split(X_train_val, y_train_val))

X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]

train_df = pd.DataFrame({'text': X_train, 'category': y_train})
val_df = pd.DataFrame({'text': X_val, 'category': y_val})
test_df = pd.DataFrame({'text': X_test, 'category': y_test})

print(f'Training set shape: {train_df.shape}')
print(f'Validation set shape: {val_df.shape}')
print(f'Test set shape: {test_df.shape}')


Training set shape: (4958, 2)
Validation set shape: (1653, 2)
Test set shape: (1653, 2)


In [10]:
print(train_df['category'].value_counts())
print(val_df['category'].value_counts())
print(test_df['category'].value_counts())

Non-anti-LGBT+ content    4319
Homophobic                 518
Transphobic                121
Name: category, dtype: int64
Non-anti-LGBT+ content    1440
Homophobic                 173
Transphobic                 40
Name: category, dtype: int64
Non-anti-LGBT+ content    1440
Homophobic                 173
Transphobic                 40
Name: category, dtype: int64


In [11]:
train_df['category'] = train_df['category'].replace('Non-anti-LGBT+ content', 'ally')
val_df['category'] = val_df['category'].replace('Non-anti-LGBT+ content', 'ally')
test_df['category'] = test_df['category'].replace('Non-anti-LGBT+ content', 'ally')

In [12]:
train_df['category'] = train_df['category'].replace('ally', 0)
train_df['category'] = train_df['category'].replace('Homophobic', 1)
train_df['category'] = train_df['category'].replace('Transphobic', 2)

In [13]:
val_df['category'] = val_df['category'].replace('ally', 0)
val_df['category'] = val_df['category'].replace('Homophobic', 1)
val_df['category'] = val_df['category'].replace('Transphobic', 2)

In [14]:
test_df['category'] = test_df['category'].replace('ally', 0)
test_df['category'] = test_df['category'].replace('Homophobic', 1)
test_df['category'] = test_df['category'].replace('Transphobic', 2)

In [15]:
train_df=train_df[["text", "category"]]
train_df = train_df.rename(columns={'text': 'text', 'category': 'labels'})
val_df=val_df[["text", "category"]]
val_df = val_df.rename(columns={'text': 'text', 'category': 'labels'})
test_df=test_df[["text", "category"]]
test_df = test_df.rename(columns={'text': 'text', 'category': 'labels'})

In [ ]:
!pip install emoji

In [18]:
import emoji
train_df['text'] = train_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
val_df['text'] = val_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
test_df['text'] = test_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

## Training Model with Englis Data

In [19]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 3,
             "max_seq_length": 128,
             "train_batch_size": 32,
             "eval_batch_size": 32,
             "logging_steps": 50,
             "save_steps": 2000,
             "learning_rate": 2e-5,
             "manual_seed": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
    num_labels=3,
    args=train_args
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [20]:
# Train the model
model.train_model(train_df)

  0%|          | 0/4958 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/155 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/155 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/155 [00:00<?, ?it/s]

(465, 0.329948447612665)

Validation Set


In [21]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')
result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score)
result

  0%|          | 0/1653 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/52 [00:00<?, ?it/s]

{'mcc': 0.5087710146670061,
 'f1': 0.6355975385970897,
 'acc': 0.8947368421052632,
 'eval_loss': 0.29048412966613585}

Testing With Tamil Data

In [22]:
test_df

,text,labels
1173,அவள் அழகாக இருக்கிறாள்,0
790,MyNameIsRam நன்றி bro red_heart,0
2410,உரக்கச் சொல் உன் உண்மையே ...........கேட்குரவன்...,0
470,"சிஸ்ஸி, உன்னிடம் சிறந்த மனப்பான்மை, நம்பிக்கை ...",1
362,சங்கிகளை எதிர்க்கும் குயர்கள் red_heart fire,0
...,...,...
739,எனது கருத்தை விரும்பு,0
923,"முதலில், நான் ஒன்று தெரிந்து கொள்ள விரும்புகிற...",1
156,கலாச்சார சீர்கேடு ஏற்பட வாய்ப்பு உள்ளது..........,1
1269,அருமையான பேச்சு மாலினி..... லவ் யூ லாட்.... நா...,0


In [23]:
test = test_df['text'].tolist()
labels = test_df['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1653 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"சிஸ்ஸி, உன்னிடம் சிறந்த மனப்பான்மை, நம்பிக்கை ...",1,0
1,சோதோம் & கொமோராவின் பாவம்,1,0
2,இவர்களுக்கும் பொதுமக்களை போல் தண்டனை சட்டம் கொ...,0,1
3,ஊர் ஓரினச்சேர்க்கை காதலாக இருந்தாலும் அது காதல...,0,1
4,யாரெல்லாம் கடவுள் ஆணுக்கு பெண்ண படச்சார் -னு ச...,0,1
...,...,...,...
161,சுத்தி நிக்கற அத்தன பேரும் சோ்ந்தா இதுகள மெரட...,0,2
162,Anchor ku பேச தெரியல ....,1,0
163,"நான் ஓரினச்சேர்க்கையில் உடன்படவில்லை, ஏனென்றால...",1,0
164,ஓரினச்சேர்க்கை இயற்கைக்கு மாறானது என்றாலும் தி...,1,0


In [24]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.94      0.96      0.95      1440
  homophobic       0.62      0.54      0.57       173
 transphobic       0.52      0.35      0.42        40

    accuracy                           0.90      1653
   macro avg       0.69      0.62      0.65      1653
weighted avg       0.89      0.90      0.90      1653



# Training with Tamil and testing with english

In [29]:
o_test=pd.read_csv('/content/tam_eng_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
e_test = e_test.rename(columns={'text                        ': 'text'})
#e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [30]:
test_ml = pd.concat([o_test,e_test])
test_ml

,Unnamed: 0,text,category,ln
0,0.0,When each commanta was read and brought up.,Non-anti-LGBT+ content,tam_eng
1,1.0,Bro live a prosperous bouquet OK_hand OK_hand ...,Non-anti-LGBT+ content,tam_eng
2,2.0,Jesus is a step sister,Non-anti-LGBT+ content,tam_eng
3,3.0,Eyes are confused....,Non-anti-LGBT+ content,tam_eng
4,4.0,Come together Gay mean not sex it's love also ...,Non-anti-LGBT+ content,tam_eng
...,...,...,...,...
985,NaN,Looks like Karthik took advantage on Varun whe...,Non-anti-LGBT+ content,NaN
986,NaN,i am really crying pro😢😢😢😭😭😭😭😭😭 ...,Non-anti-LGBT+ content,NaN
987,NaN,They may be transgender but don't ever forgot ...,Non-anti-LGBT+ content,NaN
988,NaN,It is their own choice. I support them No doub...,Non-anti-LGBT+ content,NaN


In [31]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [32]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [33]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [34]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,When each commanta was read and brought up.,0,2
1,Gay ah they are not wrong...it is created by L...,0,1
2,pranesh G you are a transgender.,2,0
3,Let natural calamities like Tsunami and Earthq...,1,0
4,Do the people in your house agree to this type...,1,0
...,...,...,...
227,"I'm sharing my experience pls read it, two tra...",2,0
228,"First of all, i wanna know onething. Do you th...",1,0
229,madam nalini....u talking too much right....ok...,1,0
230,Friendship s friendship.dont make it as sexual...,1,0


In [35]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.88      0.99      0.93      1579
  homophobic       0.64      0.12      0.20       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.51      0.37      0.38      1819
weighted avg       0.83      0.87      0.83      1819



# Training with Tamil and testing with Hindi

In [36]:
o_test=pd.read_csv('/content/eng_hin_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_hin_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [37]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [38]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [39]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [40]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [41]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,यह विशुद्ध रूप से जैविक है!,0,1
1,"आप दोनों जिन्होंने मेरी टिप्पणी का जवाब दिया, ...",1,0
2,"सत्य नारायणन , इसे आज भी पश्चिमी दुनिया में प्...",1,0
3,मुझे लगता है..यह मानसिक बीमारी है। जाओ और डॉक्...,1,0
4,सिर्फ इसलिए कि इसे कानूनी बना दिया गया है इसका...,0,1
...,...,...,...
238,वह बहुत अच्छा काम करती है। मांचूर में डेढ़ साल...,2,0
239,मैं आपकी राय मानता हूं। अगर समलैंगिकता गलत है ...,1,0
240,इसी महीने कुत्तों को देखते ही गोली मार दी गई,2,1
241,सुंदर वाणी मधुर वाणी। सवाल का जवाब बहुत अच्छा ...,1,0


In [42]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.88      0.98      0.93      1579
  homophobic       0.46      0.11      0.18       194
 transphobic       0.00      0.00      0.00        46

    accuracy                           0.87      1819
   macro avg       0.45      0.36      0.37      1819
weighted avg       0.81      0.87      0.82      1819



# Training with Tamil and testing with Telugu

In [43]:
o_test=pd.read_csv('/content/eng_tel_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_tel_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [44]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [45]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [46]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [47]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [48]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,ఇది పూర్తిగా జీవసంబంధమైనది!,0,1
1,నా వ్యాఖ్యకు ప్రత్యుత్తరం ఇచ్చిన మీరిద్దరూ దీన...,1,0
2,"సత్య నారాయణన్ , నేటికీ పాశ్చాత్య ప్రపంచంలో దీన...",1,0
3,నా అభిప్రాయం .. ఇది మానసిక వ్యాధి. వెళ్లి వైద్...,1,0
4,ఇది చట్టబద్ధం చేయబడినందున అది సరైనదని అర్థం కాదు,0,1
...,...,...,...
240,"అనాథలు మీలాగా లెస్బియన్‌గా, గేలుగా ఎందుకు పెరు...",1,0
241,మీరు తెలివిగా మాట్లాడాలని అనుకుంటున్నారు...,1,0
242,ఆమె చాలా చక్కగా నటిస్తుంది. మంచూర్‌లో డబ్బులు ...,2,0
243,నేను మీ అభిప్రాయాన్ని అంగీకరిస్తున్నాను. స్వలి...,1,0


In [49]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.88      0.98      0.93      1579
  homophobic       0.45      0.12      0.19       194
 transphobic       0.33      0.07      0.11        46

    accuracy                           0.87      1819
   macro avg       0.55      0.39      0.41      1819
weighted avg       0.82      0.87      0.83      1819



# Training with Tamil and testing with Malyalam

In [50]:
o_test=pd.read_csv('/content/eng_ml_test.tsv', delimiter='\t') 
o_test = o_test.dropna()
e_test=pd.read_csv('/content/tam_ml_test.tsv', delimiter='\t')
e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()

In [51]:
test_ml = pd.concat([o_test,e_test])
len(test_ml)

1819

In [52]:
test_ml['category'] = test_ml['category'].replace('Non-anti-LGBT+ content', 'ally')
test_ml['category'] = test_ml['category'].replace('ally', 0)
test_ml['category'] = test_ml['category'].replace('Homophobic', 1)
test_ml['category'] = test_ml['category'].replace('Transphobic', 2)

In [53]:
test_ml=test_ml[["text", "category"]]
test_ml = test_ml.rename(columns={'text': 'text', 'category': 'labels'})

In [54]:
test_ml['text'] = test_ml['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [55]:
test = test_ml['text'].tolist()
labels = test_ml['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/1819 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,ഇത് തികച്ചും ജൈവികമാണ്!,0,1
1,"സഹോദരി വിഷമിക്കരുത്, ദൈവം നിങ്ങളെ അനുഗ്രഹിക്കട...",0,1
2,എന്റെ അഭിപ്രായത്തിന് മറുപടി നൽകിയ നിങ്ങൾ രണ്ടു...,1,0
3,"സത്യ നാരായണൻ , ഇന്നും പാശ്ചാത്യലോകത്ത് ഇത് പ്ര...",1,0
4,എനിക്ക് തോന്നുന്നു .. മാനസിക രോഗമാണ്. പോയി ഡോക...,1,0
...,...,...,...
250,അവൾ വളരെ ഭംഗിയായി അഭിനയിക്കുന്നു. മഞ്ചൂരിൽ സർക...,2,0
251,താങ്കളുടെ അഭിപ്രായം ഞാൻ അംഗീകരിക്കുന്നു. സ്വവർ...,1,0
252,നല്ല സംസാരം.മധുരമായ ശബ്ദം. എന്ന ചോദ്യത്തിനുള്ള...,1,0
253,അതുകൊണ്ടാണ് നിങ്ങളുടെ ദൈവം നിങ്ങൾക്ക് ഈ വികാരം...,0,1


In [56]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.89      0.96      0.92      1579
  homophobic       0.41      0.23      0.29       194
 transphobic       0.50      0.09      0.15        46

    accuracy                           0.86      1819
   macro avg       0.60      0.42      0.45      1819
weighted avg       0.83      0.86      0.84      1819

